# Glider Speed polars analysis notebook

In [12]:
import numpy as np
import math
from numpy.polynomial import Polynomial

print(np.__version__)

# https://www.geeksforgeeks.org/python-implementation-of-polynomial-regression/?ref=lbp
# old vs new API https://numpy.org/doc/1.21/reference/routines.polynomials.html

1.21.1


In [13]:
# utilities functions & constants

KM_TO_MS = 3.6		# factor to convert km/h in m/s
CONVERT_TO_MS = False

POLAR_CURVE_START_KM = 70		# in km/h
POLAR_CURVE_END_KM = 230		# in km/h

POLAR_CURVE_START = (POLAR_CURVE_START_KM / KM_TO_MS) if CONVERT_TO_MS else POLAR_CURVE_START_KM
POLAR_CURVE_END = (POLAR_CURVE_END_KM / KM_TO_MS)if CONVERT_TO_MS else POLAR_CURVE_END_KM

#POLAR_CURVE_NBR_SAMPLE = round((POLAR_CURVE_END - POLAR_CURVE_START) // 10)
POLAR_CURVE_NBR_SAMPLE =  33

def polar_to_string(p):
	polar_str = '{}x<sup>2</sup>'.format(round(p.coeffs[0],4))
	polar_str += (' + ' if (p.coeffs[1] >= 0 ) else ' ')
	polar_str += '{}x'.format(round(p.coeffs[1],4))
	polar_str += (' + ' if (p.coeffs[2] >= 0 ) else ' ')
	polar_str += '{}'.format(round(p.coeffs[2],4))
	return polar_str

xaxis_unit = lambda x: x/ (KM_TO_MS if CONVERT_TO_MS else 1)

In [14]:

## Polar data fimport from json file
import json
glider_pdata = None	# glider polars data

# Opening JSON file
with open('./ventus2-15m-polars-db.json') as json_file:
	glider_pdata = json.load(json_file)

byhand_data = glider_pdata['specifications']['SandroExcel-m1']
abc_ref_data = glider_pdata['specifications']['SandroExcel-m2']
abc_data = glider_pdata['specifications']['SandroExcel-m4']

In [15]:


# c1 curve will be the calculated polar using the 3 points method, 
# the 3 points are estimated based on the flight polar diagram of the manual, page 5.4

c1_x = [xaxis_unit(x) for x in byhand_data['speed']]
c1_y = byhand_data['sink_rate']
c1_wl = byhand_data['wing_loading']
c1_tn = 'Manual (weight: {} kg)'.format(byhand_data['weight'])

In [16]:
# To plot ABC polar y = - (Ax^2 + Bx + C)
c2_polar = np.poly1d([abc_ref_data["A"],abc_ref_data["B"], abc_ref_data["C"] ])
c2_polar = np.polymul(c2_polar,-1)
print ('Polynomial (weight {}kg) is'.format(abc_ref_data["weight"]))
print(c2_polar) 

c2_x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)
print('\r\nVitesse (km/h)= {}'.format(c2_x_polar))

c2_y_polar = c2_polar(np.divide(c2_x_polar,100))
print('\r\nSink rate (m/s) = {}'.format(c2_y_polar))

# f = lambda x: -1*(abc_ref_data["A"]*(x/100)**2 + abc_ref_data["B"]*(x/100) + abc_ref_data["C"])
# c2_y_polar = [f(x) for x in c2_x_polar]
# print('\r\nc2_y_polar = {}'.format(c2_y_polar))

c2_wl = abc_ref_data['wing_loading']
c2_w = abc_ref_data["weight"]
c2_tn = 'Model ABC (weight: {} kg)'.format(c2_w)

c2_ld = np.divide(np.divide(-c2_x_polar , KM_TO_MS), c2_y_polar)
print('\r\nL/D = {}'.format(c2_ld))

Polynomial (weight 430kg) is
      2
-1.5 x + 2.92 x - 2.1

Vitesse (km/h)= [ 70.  75.  80.  85.  90.  95. 100. 105. 110. 115. 120. 125. 130. 135.
 140. 145. 150. 155. 160. 165. 170. 175. 180. 185. 190. 195. 200. 205.
 210. 215. 220. 225. 230.]

Sink rate (m/s) = [-0.791   -0.75375 -0.724   -0.70175 -0.687   -0.67975 -0.68    -0.68775
 -0.703   -0.72575 -0.756   -0.79375 -0.839   -0.89175 -0.952   -1.01975
 -1.095   -1.17775 -1.268   -1.36575 -1.471   -1.58375 -1.704   -1.83175
 -1.967   -2.10975 -2.26    -2.41775 -2.583   -2.75575 -2.936   -3.12375
 -3.319  ]

L/D = [24.58210423 27.63957988 30.6936771  33.64604362 36.39010189 38.82146214
 40.8496732  42.40882103 43.46451715 44.01576913 44.09171076 43.74453193
 43.04065687 42.05214466 40.8496732  39.49769824 38.05175038 36.55746598
 35.05082369 33.55909451 32.10212252 30.6936771  29.342723   28.05453194
 26.83161046 25.674448   24.58210423 23.5526603  22.58355917 21.67185783
 20.81441114 20.0080032  19.24943926]


In [17]:
# To plot ABC polar y = - (Ax^2 + Bx + C)
c3_polar = np.poly1d([abc_data["A"],abc_data["B"], abc_data["C"] ])
c3_polar = np.polymul(c3_polar,-1)
print ('Polynom with legacy API is')
print(c3_polar) 

c3_x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)

c3_y_polar = c3_polar(np.divide(c3_x_polar,100))
print('\r\nc3_y_polar = {}'.format(c3_y_polar))

c3_wl = abc_data['wing_loading']
c3_tn = 'Model ABC (weight: {} kg)'.format(abc_data['weight'])

Polynom with legacy API is
        2
-1.391 x + 2.92 x - 2.264

c3_y_polar = [-0.90209   -0.8569375 -0.81874   -0.7874975 -0.76321   -0.7458775
 -0.7355    -0.7320775 -0.73561   -0.7460975 -0.76354   -0.7879375
 -0.81929   -0.8575975 -0.90286   -0.9550775 -1.01425   -1.0803775
 -1.15346   -1.2334975 -1.32049   -1.4144375 -1.51534   -1.6231975
 -1.73801   -1.8597775 -1.9885    -2.1241775 -2.26681   -2.4163975
 -2.57294   -2.7364375 -2.90689  ]


In [18]:
# calculate polar for a specific weight, référence is c2
#
new_weight = 330

c4_x_polar = [ vi * math.sqrt(new_weight/c2_w) for vi in c2_x_polar]
c4_y_polar = [ vz * math.sqrt(new_weight/c2_w) for vz in c2_y_polar]

c4_tn = 'Adjusted (weight: {} kg)'.format(new_weight)

In [19]:
# Display the graph
# https://plotly.com/python/plotly-fundamentals/
# https://plotly.com/python-api-reference/

import plotly.graph_objects as go

# trace1 = go.Scatter( x=c1_x, y=c1_y, mode='markers', name='A, B, C points', 
# 	text=['Point A', 'Point B', 'Point C'], hovertemplate='<extra></extra><b>%{text}</b><br>Speed: %{x}km/h<br>Sink rate: %{y}m/s', hoverinfo='x+y+text')
	
trace1 = go.Scatter( x=c1_x, y=c1_y, mode='lines+markers', name=c1_tn,
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y')

trace2 = go.Scatter( x=c2_x_polar, y=c2_y_polar, mode='lines+markers', name=c2_tn, marker_symbol = 'square',marker_size=6,
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y')

trace3 = go.Scatter( x=c3_x_polar, y=c3_y_polar, mode='lines+markers', name=c3_tn, marker_symbol = 'diamond',marker_size=6,
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y')

trace4 = go.Scatter( x=c4_x_polar, y=c4_y_polar, mode='lines+markers', name=c4_tn, marker_symbol = 'x',marker_size=8,
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y')

layout = go.Layout(
		title_text='<b>{}  - Speed polars analysis</b>'.format(glider_pdata['glider']),
		height=800,
		yaxis=dict(range=[-4, 1]),
		template='ggplot2',
	)

data = [trace1, trace2, trace3, trace4]

fig = go.Figure(data=data, layout=layout)

fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black',  dtick=0.5)
fig.update_yaxes(title_text='Vz (m/s)', title_font=dict(size=14, family='Courier', color='crimson'))

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')
fig.update_xaxes(title_text='Vitesse ({})'.format(( 'm/s' if CONVERT_TO_MS else 'km/h')),
	 title_font=dict(size=14, family='Courier', color='crimson'))

# fig.add_annotation(x=xaxis_unit(50), y=-2.75, text='Polar: {}'.format( polar_to_string(c1_polar)), showarrow=False)

fig.update_layout(xaxis_range=[0,max(c1_x)*1.2])

fig.show()

In [20]:
# https://ipywidgets.readthedocs.io/en/latest/index.html
# https://github.com/jonmmease/plotly_ipywidget_notebooks

from ipywidgets import widgets, Layout, Label
import ipywidgets as iw

def polarAdjusted4Weight(new_weight):
	c4_x_polar = [ vi * math.sqrt(new_weight/c2_w) for vi in c2_x_polar]
	c4_y_polar = [ vz * math.sqrt(new_weight/c2_w) for vz in c2_y_polar]

	c4_tn = 'Adjusted (weight: {} kg)'.format(new_weight)
	return c4_x_polar, c4_y_polar, c4_tn


f2 = go.FigureWidget(fig)
# f2

weights = widgets.IntSlider(
	value=new_weight,
	min=300,
	max=550,
	step=10,
	description='Weight',
	continuous_update=True,
	readout=False 
)
weight_text = Label(value=f'{weights.value:d} kg', description='', layout=Layout(width='80px'))

container = widgets.HBox(children=[weights,weight_text,], layout=Layout(margin='0px 0px 100px 100px'))


def on_value_change(change):
	weight_text.value = f'{weights.value:d} kg'
	with f2.batch_update():
		x, y, t = polarAdjusted4Weight(weights.value)
		f2.data[3].x = x
		f2.data[3].y = y
		f2.data[3].name = t

weights.observe(on_value_change, names="value")

widgets.VBox([f2, container])



    'data': [{'hoverinfo': 'x+y',
              'hovertemplate': '<extra></extra…